### Culture Knowledge Graphs Using MELODY and the public SPARQL Endpoint for NFDI4Culture

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import json

In [ ]:
with open("../files/queries.json") as f:
    queries = json.load(f)

In [ ]:
queries = queries["feasible_queries"]

In [ ]:
queries

In [ ]:
sparql = SPARQLWrapper("https://nfdi4culture.de/sparql")
sparql.setReturnFormat(JSON)

In [ ]:
QUERIES = {
    "n_triplets": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT (COUNT(*) AS ?tripleCount)
        WHERE {
            ?s ?p ?o .
        }
    """,

    "all_types": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?type
        WHERE {
            ?s rdf:type ?type .
        }
        ORDER BY ?type
    """,

    "n_instances_per_type": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?type (COUNT(?s) AS ?instanceCount)
        WHERE {
            ?s rdf:type ?type .
        }
        GROUP BY ?type
        ORDER BY DESC(?instanceCount)
    """,

    "type_properties": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?type ?property (COUNT(?property) AS ?propertyCount)
        WHERE {
            ?s rdf:type ?type ;
               ?property ?o .
        }
        GROUP BY ?type ?property
        ORDER BY ?type DESC(?propertyCount)
    """,

    "all_properties": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?property
        WHERE {
            ?s ?property ?o .
        }
        ORDER BY ?property
    """,

    "n_instances_per_property": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?property (COUNT(?s) AS ?instanceCount)
        WHERE {
            ?s ?property ?o .
        }
        GROUP BY ?property
        ORDER BY DESC(?instanceCount)
    """,

    "property_values": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?property ?value (COUNT(?s) AS ?usageCount)
        WHERE {
            ?s ?property ?value .
        }
        GROUP BY ?property ?value
        ORDER BY ?property DESC(?usageCount)
    """,

    "subjects_with_property": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?subject
        WHERE {
            ?subject ?property ?o .
            FILTER(?property = <PROPERTY_URI>)
        }
        ORDER BY ?subject
    """,

    "objects_with_property": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?object
        WHERE {
            ?s ?property ?object .
            FILTER(?property = <PROPERTY_URI>)
        }
        ORDER BY ?object
    """,

    "subjects_with_type": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?subject
        WHERE {
            ?subject rdf:type <TYPE_URI> .
        }
        ORDER BY ?subject
    """,

    "objects_with_type": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?object
        WHERE {
            ?s ?p ?object .
            ?object rdf:type <TYPE_URI> .
        }
        ORDER BY ?object
    """,

    "subjects_with_property_value": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?subject
        WHERE {
            ?subject ?property <VALUE_URI> .
            FILTER(?property = <PROPERTY_URI>)
        }
        ORDER BY ?subject
    """,

    "objects_with_property_value": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?object
        WHERE {
            ?s ?property ?object .
            FILTER(?property = <PROPERTY_URI> && ?object = <VALUE_URI>)
        }
        ORDER BY ?object
    """,

    "subjects_with_type_and_property": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?subject
        WHERE {
            ?subject rdf:type <TYPE_URI> ;
                     ?property ?o .
            FILTER(?property = <PROPERTY_URI>)
        }
        ORDER BY ?subject
    """,

    "objects_with_type_and_property": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?object
        WHERE {
            ?s ?property ?object .
            ?object rdf:type <TYPE_URI> .
            FILTER(?property = <PROPERTY_URI>)
        }
        ORDER BY ?object
    """,

    "subjects_with_type_and_property_value": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?subject
        WHERE {
            ?subject rdf:type <TYPE_URI> ;
                     ?property <VALUE_URI> .
            FILTER(?property = <PROPERTY_URI>)
        }
        ORDER BY ?subject
    """,

    "objects_with_type_and_property_value": """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?object
        WHERE {
            ?s ?property ?object .
            ?object rdf:type <TYPE_URI> .
            FILTER(?property = <PROPERTY_URI> && ?object = <VALUE_URI>)
        }
        ORDER BY ?object
    """
}

In [ ]:
sparql.setQuery(queries["count_compositions_composer_name_decade"])

In [ ]:
try:
    ret = sparql.queryAndConvert()
except Exception as e:
    print(e)

In [ ]:
df = pd.DataFrame(ret["results"]["bindings"])
# for each entry, conver entry dict into dict['value']
df = df.applymap(lambda x: x['value'])

In [ ]:
df

In [ ]:
bindings = ret["results"]["bindings"]
rows = [{'sub': f"<{b['sub']['value']}>",
         'pred': 'rdf:type' if 'rdf-syntax-ns#type' in b['pred']['value'] else b['pred']['value'],
         'obj': f"<{b['obj']['value']}>"}
        for b in bindings]
df = pd.DataFrame(rows)

In [ ]:
df